In [36]:
import pandas as pd
import numpy as np

In [37]:
article_data= pd.read_csv('article.csv')
article_data.head()

,Title,Description,Link,Category,Publication Date,article_id
0,What India’s Jewish buildings have to say,Jewish-origin architecture in the country appe...,https://www.thehindu.com/society/history-and-c...,life-and-style,"Fri, 05 Apr 2024 13:30:03 +0530",1
1,Chef Amninder Sandhu’s Bawri in Goa is all abo...,Amninder Sandhu is using open-fire cooking and...,https://www.thehindu.com/life-and-style/food/c...,life-and-style,"Fri, 04 Aug 2023 15:51:18 +0530",2
2,Palestinian athletes to be invited to Paris Ol...,The IOC head dismissed suggestions the IOC has...,https://www.thehindu.com/sport/palestinian-ath...,Sport,"Sat, 27 Apr 2024 11:50:11 +0530",4
3,"Interview | Call him old or call him a find, S...",The lad from Bhilai made his debut in senior d...,https://www.thehindu.com/sport/cricket/intervi...,Sport,"Thu, 16 May 2024 00:41:12 +0530",5
4,"Saif Ali Khan, Siddharth Anand reuniting for n...",Anand and Khan had worked together on films li...,https://www.thehindu.com/entertainment/movies/...,Entertainment,"Mon, 27 May 2024 12:14:27 +0530",6


In [38]:
# article_data['Publication Date'] = pd.to_datetime(article_data['Publication Date'], format='%a, %d %b %Y %H:%M:%S %z')

# # Sort DataFrame by 'Publication Date'
# article_data= article_data.sort_values(by='Publication Date')

In [39]:
article_data.tail()

,Title,Description,Link,Category,Publication Date,article_id
2437,Zee seeks $90 million termination fee from Son...,It has sought termination fees from two Sony G...,https://www.thehindu.com/business/Industry/zee...,Business,"Fri, 24 May 2024 13:23:37 +0530",2895
2438,Govt. should announce legally backed minimum p...,Karnataka Agriculture Price Commission headed ...,https://www.thehindu.com/news/cities/Mangalore...,Business,"Sat, 20 Apr 2024 19:11:43 +0530",2896
2439,Madras Players’ Kamalakshi weaves together mus...,"Directed by PC Ramakrishna, and starring Sikki...",https://www.thehindu.com/entertainment/theatre...,Entertainment,"Tue, 27 Feb 2024 12:54:23 +0530",2897
2440,‘IF’ movie review: John Krasinski’s sweet ode ...,"Despite its occasional flashes of sweetness, t...",https://www.thehindu.com/entertainment/movies/...,Entertainment,"Fri, 17 May 2024 16:42:40 +0530",2898
2441,"Breast cancer awareness month: Stay vigilant, ...",October marks breast cancer awareness month. H...,https://www.thehindu.com/life-and-style/octobe...,life-and-style,"Thu, 27 Oct 2022 16:21:16 +0530",2899


In [40]:
article_data.shape

(2442, 6)

In [41]:
null_ct_count = article_data['Category'].isnull().sum()
print("Number of null category:", null_ct_count)

Number of null category: 0


In [42]:
user_data= pd.read_csv('clickstream_data.csv')
user_data.head()

,UserId,SessionId,ArticleId_served,Click,Time_Spent
0,1,1_1,2856,Yes,232
1,1,1_1,1249,Yes,167
2,1,1_1,285,Yes,50
3,1,1_1,550,Yes,36
4,1,1_1,2521,No,0


In [43]:
user_data.tail()

,UserId,SessionId,ArticleId_served,Click,Time_Spent
16577,1000,1000_1,1966,Yes,271
16578,1000,1000_1,2660,Yes,45
16579,1000,1000_2,76,Yes,30
16580,1000,1000_2,1454,Yes,243
16581,1000,1000_2,1106,Yes,22


In [44]:
user_data.shape

(16582, 5)

Vectorizing using Word2Vec

In [45]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [46]:
# sentences = article_data['Description'].apply(lambda x: simple_preprocess(x)).tolist()
# word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# # Function to get article vector
# def get_article_vector(text, model):
#     words = simple_preprocess(text)
#     word_vectors = [model.wv[word] for word in words if word in model.wv]
#     return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(model.vector_size)

# # Create article vectors
# article_data['Vector'] = article_data['Description'].apply(lambda x: get_article_vector(x, word2vec_model))
# article_vectors = np.vstack(article_data['Vector'].values)

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [49]:
# Create a mapping from ArticleId to row index
article_id_to_index = {article_id: idx for idx, article_id in enumerate(article_data['article_id'])}


Using tf-idf

In [50]:
article_vectors = tfidf_vectorizer.fit_transform(article_data['Description'].values.astype('U'))


In [51]:
# Initialize a dictionary to hold user profiles
user_profiles = {}

In [52]:
# Generate user profiles
for user_id, group in user_data.groupby('UserId'):
    article_ids = group.loc[group['Click'] == 'Yes', 'ArticleId_served']
    valid_indices = [article_id_to_index[article_id] for article_id in article_ids if article_id in article_id_to_index]
    if valid_indices:
        user_profile = article_vectors[valid_indices].mean(axis=0)
        user_profiles[user_id] = user_profile

In [53]:
def recommend_articles(user_id, num_recommendations=10):
    user_profile = user_profiles.get(user_id)
    if user_profile is None:
        print(f"User with ID {user_id} does not exist.")
        return
    
    # Convert user_profile to a numpy array
    user_profile = np.asarray(user_profile)
    
    # Calculate similarity scores between the user profile and all articles
    similarity_scores = cosine_similarity(user_profile, article_vectors).flatten()
    
    # Get indices of top similar articles
    top_indices = similarity_scores.argsort()[-num_recommendations:][::-1]
    
    # Get recommended articles with only the desired columns
    recommended_articles = article_data.iloc[top_indices][['article_id', 'Category', 'Title', 'Description','Link']]
    
    return recommended_articles



In [54]:
# Merge user_data with article_data to get article information
user_clicks_info = pd.merge(user_data[['UserId', 'ArticleId_served', 'Time_Spent']], 
                            article_data[['article_id', 'Category']], 
                            left_on='ArticleId_served', 
                            right_on='article_id', 
                            how='left')

# Drop the redundant column
user_clicks_info.drop(columns=['article_id'], inplace=True)

# Display user clicks with article information
print(user_clicks_info)


       UserId  ArticleId_served  Time_Spent        Category
0           1              2856         232   Entertainment
1           1              1249         167  life-and-style
2           1               285          50         Science
3           1               550          36         Science
4           1              2521           0        Business
...       ...               ...         ...             ...
16577    1000              1966         271   Entertainment
16578    1000              2660          45   Entertainment
16579    1000                76          30   Entertainment
16580    1000              1454         243        Business
16581    1000              1106          22           Sport

[16582 rows x 4 columns]


In [55]:
# when vectorizing using tf-idf
user_id = 23

user_clicks_info_for_user = user_clicks_info[(user_clicks_info['UserId'] == user_id) & (user_clicks_info['Time_Spent'] != 0)]

print("User clicks information with non-zero time spent:")
print(user_clicks_info_for_user)

recommended_articles_for_user = recommend_articles(user_id, num_recommendations=5) 
print("\nRecommended articles:")
print(recommended_articles_for_user)


User clicks information with non-zero time spent:
     UserId  ArticleId_served  Time_Spent        Category
395      23              1180         257   Entertainment
396      23               664         270        Business
397      23               948         263           Sport
398      23              2292         165  life-and-style
399      23              2054          40           World
400      23               755         287           Sport
401      23              2890         185           Sport
402      23              1549         287   Entertainment
404      23              2405         100        Business
405      23              1265         299           Sport

Recommended articles:
      article_id        Category  \
2433        2890           Sport   
1072        1265           Sport   
1933        2292  life-and-style   
798          948           Sport   
1732        2054           World   

                                                  Title  \
2433  Sunil C

In [56]:
# # when vectorizing using word2vec
# user_id = 23

# user_clicks_info_for_user = user_clicks_info[(user_clicks_info['UserId'] == user_id) & (user_clicks_info['Time_Spent'] != 0)]

# print("User clicks information with non-zero time spent:")
# print(user_clicks_info_for_user)

# recommended_articles_for_user = recommend_articles(user_id, num_recommendations=5) 
# print("\nRecommended articles:")
# print(recommended_articles_for_user)


In [57]:
recommended_articles = recommend_articles(user_id=3, num_recommendations=10)
print(recommended_articles)

      article_id        Category  \
2398        2847  life-and-style   
2310        2748        Business   
1267        1485   Entertainment   
1912        2269   Entertainment   
1036        1225           Sport   
2225        2638  life-and-style   
2247        2664   Entertainment   
398          483  life-and-style   
979         1158  life-and-style   
1299        1525  life-and-style   

                                                  Title  \
2398                                 All roads to Milan   
2310  Interim Budget 2024 | Thrust for PM Vishwakarm...   
1267  ‘Turbo’ movie review: Mammootty’s charge held ...   
1912  ‘Turbo’ movie review: Mammootty’s charge held ...   
1036            Junior Davis Cup selection camp at DLTA   
2225  The growing industry of personalised beauty br...   
2247  Maneesh Verma: ‘Jump is about nihilism versus ...   
398   Indian Pursuit, Chieftain Elite variants unveiled   
979           What is DOMS, and how do we deal with it?   
1299         

In [58]:
# from tkinter import *
# # Function to format text for display
# def every_100(text):
#     final_text = ""
#     for i in range(len(text)):
#         final_text += text[i]
#         if (i + 1) % 100 == 0:
#             final_text += "\n"
#     return final_text

# # Create the Tkinter root window
# root = Tk()
# root.title("Recommended Articles")
# root.geometry("1000x800")
# root.configure(bg='#f0f0f0')  # Set a light gray background color

# if recommended_articles is not None:
#     # Create a scrollbar
#     scrollbar = Scrollbar(root)
#     scrollbar.pack(side=RIGHT, fill=Y)

#     # Create a canvas and connect it to the scrollbar
#     canvas = Canvas(root, yscrollcommand=scrollbar.set, bg='#f0f0f0')
#     canvas.pack(side=LEFT, fill=BOTH, expand=True)
#     scrollbar.config(command=canvas.yview)

#     # Create a frame within the canvas
#     frame = Frame(canvas, bg='#f0f0f0')
#     canvas.create_window((0, 0), window=frame, anchor='nw')

#     # Function to update the scroll region
#     def on_configure(event):
#         canvas.config(scrollregion=canvas.bbox('all'))

#     # Bind the configure event to update the scroll region
#     frame.bind('<Configure>', on_configure)

#     # Create frames and labels to display the recommendations
#     for idx, row in recommended_articles.iterrows():
#         article_frame = Frame(frame, width=900, height=200, pady=10, padx=10, bg='white', bd=2, relief='groove')
        
#         title_label = Label(article_frame, text=row['Title'], font=("Helvetica", 16, "bold"), fg="#337833", bg='white', wraplength=850, justify=LEFT, padx=10, pady=5)
#         title_label.pack(anchor="w")
        
#         description_label = Label(article_frame, text=every_100(row['Description']), font=("Helvetica", 12), fg="#532555", bg='white', wraplength=850, justify=LEFT, padx=10, pady=5)
#         description_label.pack(anchor="w")
        
#         article_frame.pack(anchor="w", pady=5, padx=10)

# root.mainloop()

In [59]:
from tkinter import *
import pandas as pd
import webbrowser
import datetime
import time
import os

# Global variable to track the start time and article ID
start_time = None
current_article_id = None
current_link = None

# Function to format text for display
def every_100(text):
    final_text = ""
    for i in range(len(text)):
        final_text += text[i]
        if (i + 1) % 100 == 0:
            final_text += "\n"
    return final_text

# Function to open the article link in a web browser and log clickstream data
def open_article(user_id, article_id, link):
    global start_time, current_article_id, current_link
    start_time = time.time()  # Start the timer
    current_article_id = article_id
    current_link = link
    log_clickstream(user_id, article_id, link, "Yes", 0)  # Log the click with 0 time spent initially
    webbrowser.open(link)

# Function to stop the timer and log the time spent
def return_from_article(user_id):
    global start_time, current_article_id, current_link
    if start_time is not None:
        end_time = time.time()
        time_spent = int(end_time - start_time)  # Calculate the time spent
        update_clickstream(user_id, current_article_id, current_link, time_spent)  # Update the clickstream with the time spent
        start_time = None  # Reset the timer

# Function to log clickstream data
def log_clickstream(user_id, article_id, link, click, time_spent):
    timestamp = datetime.datetime.now()
    clickstream_entry = {
        'UserId': user_id,
        'ArticleId_served': article_id,
        'Click': click,
        'Time_Spent': time_spent,
        'Timestamp': timestamp
    }
    clickstream_df = pd.DataFrame([clickstream_entry])
    
    # Write header only if the file does not exist
    file_exists = os.path.isfile('interaction.csv')
    clickstream_df.to_csv('interaction.csv', mode='a', header=not file_exists, index=False)

# Function to update clickstream data with time spent
def update_clickstream(user_id, article_id, link, time_spent):
    try:
        clickstream_df = pd.read_csv('interaction.csv')
        mask = (clickstream_df['UserId'] == user_id) & (clickstream_df['ArticleId_served'] == article_id)
        clickstream_df.loc[mask, 'Time_Spent'] = time_spent
        clickstream_df.to_csv('interaction.csv', index=False)
    except pd.errors.ParserError as e:
        print(f"Error reading the CSV file: {e}")

# Create the Tkinter root window
root = Tk()
root.title("Recommended Articles")
root.geometry("1000x800")
root.configure(bg='#f0f0f0')  

user_id = 23

recommended_articles = recommend_articles(user_id=user_id)  # Make sure to call your recommendation function here

if recommended_articles is not None:
    # Create a scrollbar
    scrollbar = Scrollbar(root)
    scrollbar.pack(side=RIGHT, fill=Y)

    # Create a canvas and connect it to the scrollbar
    canvas = Canvas(root, yscrollcommand=scrollbar.set, bg='#f0f0f0')
    canvas.pack(side=LEFT, fill=BOTH, expand=True)
    scrollbar.config(command=canvas.yview)

    # Create a frame within the canvas
    frame = Frame(canvas, bg='#f0f0f0')
    canvas.create_window((0, 0), window=frame, anchor='nw')

    # Function to update the scroll region
    def on_configure(event):
        canvas.config(scrollregion=canvas.bbox('all'))

    # Bind the configure event to update the scroll region
    frame.bind('<Configure>', on_configure)

    # Create frames and labels to display the recommendations
    for idx, row in recommended_articles.iterrows():
        article_frame = Frame(frame, width=900, height=200, pady=10, padx=10, bg='white', bd=2, relief='groove')
        
        # Create a clickable title label
        title_label = Label(article_frame, text=row['Title'], font=("Helvetica", 16, "bold"), fg="#337833", bg='white', wraplength=850, justify=LEFT, padx=10, pady=5, cursor="hand2")
        title_label.pack(anchor="w")
        title_label.bind("<Button-1>", lambda e, user_id=user_id, article_id=row['article_id'], link=row['Link']: open_article(user_id, article_id, link))
        
        description_label = Label(article_frame, text=every_100(row['Description']), font=("Helvetica", 12), fg="#532555", bg='white', wraplength=850, justify=LEFT, padx=10, pady=5)
        description_label.pack(anchor="w")
        
        article_frame.pack(anchor="w", pady=5, padx=10)
    
    # Add a return button to log the time spent when returning
    return_button = Button(root, text="Return from Article", font=("Helvetica", 14), bg="#337833", fg="white",
                           command=lambda: return_from_article(user_id))
    return_button.pack(side=BOTTOM, pady=20)

root.mainloop()
